#**Abdus Samad (DHC-474)**
#**Task 1**
#News Topic Classifier using BERT

# -------------------------
# 1) INSTALL & IMPORTS
# -------------------------

In [1]:
!pip install -q transformers datasets evaluate gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import pandas as pd
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, TrainingArguments, Trainer
)
import torch
from sklearn.metrics import accuracy_score, f1_score
import gradio as gr

# -------------------------
# 2) LOAD DATASET
# -------------------------
 AG News = 4 classes (World, Sports, Business, Sci/Tech)

In [3]:
dataset = load_dataset("ag_news")

print(dataset)   # train: 120k, test: 7.6k
print(dataset["train"][0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})
{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}


# -------------------------
# 3) TOKENIZER & PREPROCESSING
# -------------------------

In [4]:
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
# Function to tokenize text
def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True)


In [6]:

# Apply tokenizer to dataset
tokenized_ds = dataset.map(tokenize_fn, batched=True)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [7]:
# Data collator = handles dynamic padding for batches
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

 -------------------------
# 4) LOAD MODEL
# -------------------------
num_labels = 4 (AG News has 4 categories)

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=4)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# -------------------------
# 5) METRICS
# -------------------------
Define evaluation metrics (Accuracy + F1)

In [9]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=preds, references=labels)
    f1 = f1_metric.compute(predictions=preds, references=labels, average="weighted")
    return {"accuracy": acc["accuracy"], "f1": f1["f1"]}

# -------------------------
# 6) TRAINING ARGUMENTS
# -------------------------

In [10]:
training_args = TrainingArguments(
    output_dir="./bert-news",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,     # increase for better performance
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    push_to_hub=False
)

# -------------------------
# 7) TRAINER
# -------------------------

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-2210026093.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


# -------------------------
# 8) TRAIN MODEL
# -------------------------

In [12]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abdussamad56403 (abdussamad56403-ned-university-of-engineering-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,0.710000
200,0.370400
300,0.340200
400,0.304600
500,0.295700
600,0.277300
700,0.275100
800,0.240500
900,0.276600
1000,0.237700


TrainOutput(global_step=15000, training_loss=0.17760731938680013, metrics={'train_runtime': 4069.9898, 'train_samples_per_second': 58.968, 'train_steps_per_second': 3.686, 'total_flos': 1.1997273780301056e+16, 'train_loss': 0.17760731938680013, 'epoch': 2.0})

# -------------------------
# 9) EVALUATE MODEL
# -------------------------


In [13]:
metrics = trainer.evaluate()
print("Final Evaluation:", metrics)

Final Evaluation: {'eval_loss': 0.1848028302192688, 'eval_accuracy': 0.9497368421052632, 'eval_f1': 0.9497952703256325, 'eval_runtime': 39.3764, 'eval_samples_per_second': 193.009, 'eval_steps_per_second': 12.063, 'epoch': 2.0}


# -------------------------
# 10) SAVE MODEL
# -------------------------

In [14]:
save_dir = "/content/bert_news_classifier"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Model saved at {save_dir}")

Model saved at /content/bert_news_classifier


# -------------------------
# 11) DEPLOY WITH GRADIO
# -------------------------

In [15]:
# Define labels for AG News dataset
id2label = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech"
}

def classify_news(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        pred_id = torch.argmax(probs, dim=-1).item()
    return {id2label[i]: float(probs[0][i]) for i in range(len(id2label))}

# Build Gradio UI
demo = gr.Interface(
    fn=classify_news,
    inputs=gr.Textbox(lines=2, placeholder="Enter a news headline here..."),
    outputs=gr.Label(num_top_classes=4),
    title="📰 News Topic Classifier (BERT)",
    description="Fine-tuned BERT on AG News dataset. Try typing a news headline!"
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://40fab96381f8ab1dad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
